<a href="https://colab.research.google.com/github/hizieun/ML-yahac/blob/main/yahac_ML_7_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BatchNormalization layer를 사용하여 보다 학습이 잘되는 모델을 만들어 봅니다.
* BatchNormalization layer : 기본적으로 Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다. 지금까지는 이 문제를 Activation 함수의 변화 (ReLU 등), Careful Initialization, small learning rate 등으로 해결하였지만, 이 논문에서는 이러한 간접적인 방법보다는 training 하는 과정 자체를 전체적으로 안정화하여 학습 속도를 가속시킬 수 있는 근본적인 방법이라고 할 수 있다. 
실제로 이 Batch Normalization을 네트워크에 적용시킬 때는, 특정 Hidden Layer에 들어가기 전에 Batch Normalization Layer를 더해주어 input을 modify해준 뒤 새로운 값을 activation function으로 넣어주는 방식으로 사용한다. 
Batch Normalization에서는 각 layer에 들어가는 input을 normalize 시킴으로써 layer의 학습을 가속하는데, 이 때 whitening 등의 방법을 쓰는 대신 각 mini-batch의 mean과 variance를 구하여 normalize한다.
* CNN에 적용 : Batch Normalization을 CNN에 적용시키고 싶을 경우 지금까지 설명한 방법과는 다소 다른 방법을 이용해야만 한다. 먼저, convolution layer에서 보통 activation function에 값을 넣기 전 Wx+b 형태로 weight를 적용시키는데, Batch Normalization을 사용하고 싶을 경우 normalize 할 때 beta 값이 b의 역할을 대체할 수 있기 때문에 b를 없애준다. 또한, CNN의 경우 convolution의 성질을 유지시키고 싶기 때문에, 각 channel을 기준으로  각각의 Batch Normalization 변수들을 만든다. 예를 들어 m의 mini-batch-size, n의 channel size 를 가진 Convolution Layer에서 Batch Normalization을 적용시킨다고 해보자. convolution을 적용한 후의 feature map의 사이즈가 p x q 일 경우, 각 채널에 대해 m x p x q 개의 각각의 스칼라 값에 대해 mean과 variance를 구하는 것이다. 최종적으로 gamma와 beta는 각 채널에 대해 한개씩 해서 총 n개의 독립적인 Batch Normalization 변수들이 생기게 된다.
* Batch Normalization의 장점 : 기존 Deep Network에서는 learning rate를 너무 높게 잡을 경우 gradient가 explode/vanish 하거나, 나쁜 local minima에 빠지는 문제가 있었다. 이는 parameter들의 scale 때문인데, Batch Normalization을 사용할 경우 propagation 할 때 parameter의 scale에 영향을 받지 않게 된다. 따라서, learning rate를 크게 잡을 수 있게 되고 이는 빠른 학습을 가능케 한다.
Batch Normalization의 경우 자체적인 regularization 효과가 있다. 이는 기존에 사용하던 weight regularization term 등을 제외할 수 있게 하며, 나아가 Dropout을 제외할 수 있게 한다 (Dropout의 효과와 Batch Normalization의 효과가 같기 때문.) . Dropout의 경우 효과는 좋지만 학습 속도가 다소 느려진다는 단점이 있는데, 이를 제거함으로서 학습 속도도 향상된다.
(참고 : https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/ )

1. 보스턴 집값 예측

In [3]:
# 라이브러리 사용
import tensorflow as tf
import pandas as pd

##########################
# 1. 과거의 데이터 준비
file_path = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/boston.csv'
boston = pd.read_csv(file_path)

# 종속변수, 독립변수
var_x = boston[['crim', 'zn', 'indus', 'chas', 'nox', 
            'rm', 'age', 'dis', 'rad', 'tax',
            'ptratio', 'b', 'lstat']]
var_y = boston[['medv']]
print(var_x.shape, var_y.shape)

##########################
# 2. 모델의 구조 만들기
X = tf.keras.layers.Input(shape=[13])
H = tf.keras.layers.Dense(8, activation='swish')(X)
H = tf.keras.layers.Dense(8, activation='swish')(H)
H = tf.keras.layers.Dense(8, activation='swish')(H)
Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='mse')

# 2. 모델의 구조를 BatchNormalization layer를 사용하여 만들기
X = tf.keras.layers.Input(shape=[13])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='mse')

#############################
# 3. 데이터로 모델 학습(FIT)
model.fit(var_x, var_y, epochs=1000)


(506, 13) (506, 1)
Epoch 1/1000
16/16 [==============================] - 1s 1ms/step - loss: 571.0808
Epoch 2/1000
16/16 [==============================] - 0s 1ms/step - loss: 555.3903
Epoch 3/1000
16/16 [==============================] - 0s 2ms/step - loss: 590.4825
Epoch 4/1000
16/16 [==============================] - 0s 1ms/step - loss: 538.9230
Epoch 5/1000
16/16 [==============================] - 0s 1ms/step - loss: 571.2790
Epoch 6/1000
16/16 [==============================] - 0s 1ms/step - loss: 610.7739
Epoch 7/1000
16/16 [==============================] - 0s 1ms/step - loss: 542.3776
Epoch 8/1000
16/16 [==============================] - 0s 1ms/step - loss: 551.5101
Epoch 9/1000
16/16 [==============================] - 0s 1ms/step - loss: 552.0227
Epoch 10/1000
16/16 [==============================] - 0s 1ms/step - loss: 507.9930
Epoch 11/1000
16/16 [==============================] - 0s 1ms/step - loss: 536.6397
Epoch 12/1000
16/16 [==============================] - 0s 1ms/step

2. 아이리스 품종 분류

In [6]:
# 라이브러리 사용
import tensorflow as tf
import pandas as pd

#######################
# 1.과거의 데이터를 준비
file_path = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/iris.csv'
iris = pd.read_csv(file_path)

# 원핫인코딩
iris = pd.get_dummies(iris)

# 종속변수, 독립변수
var_x = iris[['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭']]
var_y=  iris[['품종_setosa', '품종_versicolor', '품종_virginica']]
print(var_x.shape, var_y.shape)

#######################
# 2.모델의 구조 만들기
X = tf.keras.layers.Input(shape=[4])
H = tf.keras.layers.Dense(8, activation='swish')(X)
H = tf.keras.layers.Dense(8, activation='swish')(H)
H = tf.keras.layers.Dense(8, activation='swish')(H)
Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

# 2. 모델의 구조를 BatchNormalization layer를 사용하여 만들기
X = tf.keras.layers.Input(shape=[4])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

#######################
# 3. 데이터로 모델을 학습(FIT)
model.fit(var_x, var_y, epochs=1000)

(150, 4) (150, 3)
Epoch 1/1000
5/5 [==============================] - 1s 3ms/step - loss: 1.2794 - accuracy: 0.2477
Epoch 2/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.1266 - accuracy: 0.4198
Epoch 3/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.0858 - accuracy: 0.3765
Epoch 4/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.0116 - accuracy: 0.4915
Epoch 5/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.9400 - accuracy: 0.6574
Epoch 6/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.8734 - accuracy: 0.7819
Epoch 7/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.8102 - accuracy: 0.8422
Epoch 8/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.7741 - accuracy: 0.8323
Epoch 9/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.7950 - accuracy: 0.8212
Epoch 10/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.7633 - accuracy: 0.8085
E